In [1]:
import torch

In [2]:
origin_checkpoint_path = '/mnt/cache/zhujinguo/codes/UniPerceiver/work_dirs/deepspeed_moe/BERT_L12_H768_experiments/16task_90k_bertbase_lr1e-3_wd0.2_gc0.1_prenorm_warm10k_layerscale1e-3_uniformdp0.1_maeinit_fixedpos_torchfp16_unifieddataset_changeweight_stage2_224size/bertbase_womoe_pretrain2/89999/mp_rank_00_model_states.pt'

In [3]:
origin_checkpoint = torch.load(origin_checkpoint_path, 'cpu')
origin_checkpoint.keys()
# list(origin_checkpoint['module'].keys())

dict_keys(['module', 'buffer_names', 'optimizer', 'lr_scheduler', 'sparse_tensor_module_names', 'skipped_steps', 'global_steps', 'global_samples', 'dp_world_size', 'mp_world_size', 'ds_config', 'ds_version', 'iteration'])

In [4]:
len(list(origin_checkpoint['module'].keys()))

201

In [8]:
# new_checkpoint_path = 'new_exp/model_Epoch_00160_Iter_0000159.pth'
# new_checkpoint = torch.load(new_checkpoint_path, 'cpu')
# new_checkpoint.keys()
# list(new_checkpoint['model'].keys())

dict_keys(['model', 'trainer', 'amp_scaler', 'scheduler', 'iteration'])

In [10]:
# len(list(new_checkpoint['model'].keys()))

41

In [5]:
mapping_dict = {

    'encoder.': 'fused_encoder.',
    'attention.self.qkv_proj.weight': 'self_attn.in_proj_weight',
    'attention.self.qkv_proj.bias': 'self_attn.in_proj_bias',
    'attention.output.dense': 'self_attn.out_proj',
    'attention_output.residual_scale': 'gamma_1',
    'ffn.dense.': 'linear1.',
    'ffn.dense2.': 'linear2.',
    'ffn_output.residual_scale': 'gamma_2',
    'LayerNormModules.0.': 'norm1.',
    'LayerNormModules.1.': 'norm2.',
    'predictor.': 'loss_prepare.',
    
}


In [6]:
new_checkpoint = { } 

module_checkpoint = origin_checkpoint['module']

for k, v in module_checkpoint.items():
    if k.endswith('residual_scale'):
        v.squeeze_(1).squeeze_(0)
    if k.startswith('visual_embed'):
        continue
    for origin_str, target_str in mapping_dict.items():
        if origin_str in k:
            k = k.replace(origin_str, target_str)
    
    new_checkpoint[k] = v.float()

# merge type embedding in video_embed 
new_checkpoint['video_embed.embeddings.bias'] = new_checkpoint['video_embed.embeddings.bias'] + new_checkpoint['video_embed.embeddings_type.weight'][0]

In [7]:

torch.save({ 'model': new_checkpoint}, '/mnt/lustre/zhujinguo/codes/Uni-Perceiver/work_dirs/pretrained_models/uni-perceiver-base-L12-H768-224size-pretrained.pth')